In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.polynomial.legendre as leg

Set up classes

In [173]:
class particle:
     def __init__(self, alive, mu, x, g, w):
        self.alive = alive
        self.mu = mu
        self.x = x
        self.g = g
        self.w = w

In [174]:
class source:
    def __init__(self, scatter, fission):
        self.scatter = scatter
        self.fission = fission

In [175]:
class tally:
    def __init__(self, cell_average_flux, cell_edge_flux, current, k):
        self.cell_average_flux = cell_average_flux
        self.cell_edge_flux = cell_edge_flux
        self.current = current
        self.k = k
    
    def zero_g(self, g):
        self.cell_average_flux[:, g] *= 0
        self.cell_edge_flux[:, g] *= 0
        self.current[:, g] *= 0

    def zero(self):
        self.cell_average_flux *= 0
        self.cell_edge_flux *= 0
        self.current *= 0

In [176]:
class material:
    def __init__(self, nuSigmaF, SigmaS, SigmaT):
        self.nuSigmaF = nuSigmaF
        self.SigmaS = SigmaS
        self.SigmaT = SigmaT

In [177]:
class cell:
    def __init__(self, left, right, material):
        self.left = left
        self.right = right
        self.material = material

In [178]:
class mesh:
    def __init__(self, Nx, L, bc, U, W):
        self.Nx = Nx
        self.L = L
        self.dx = L/(Nx-1)
        self.mesh_x = np.linspace(0, L, Nx)
        self.mesh_flux = np.linspace(0, L, Nx-1)
        self.bc = bc
        self.U = U
        self.W = W

In [179]:
class bc:
    def __init__(self, bc):
        self.bc = bc

Set up functions

In [180]:
def f_convergence(new, old):
    # convergence criteria
    epi = 1e-5
    if np.amax(abs(new - old)) < epi:
        return True
    else:
        return False

In [181]:
def k_convergence(new, old):
    # convergence criteria
    ek = 1e-5
    if np.amax(abs((new - old) / new)) < ek:
        return True
    else:
        return False

In [182]:
def si_convergence(new, old):
    # convergence criteria
    esi = 1e-6
    if np.amax(abs(new - old) / new) < esi:
        return True
    else:
        return False

In [183]:
def get_surface(p, bin_id):
    if p.mu > 0:
        return bin_id+1
    else:
        return bin_id

In [184]:
def get_material(p, cells):
    for i in range(len(cells)):
        right = cells[i].right
        left = cells[i].left
        if p.x >= left and p.x < right:
            return cells[i].material

In [185]:
def get_pin_average(flux):
    # get values
    dx = mesh1.dx
    hist = np.zeros([48,2])
    index = 0

    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        average0 = 0
        average1 = 0
        
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        nuSigmaF = material.nuSigmaF
        bins = int((right - left) / dx)

        # calculate source for current cell
        for j in range(bins):
            j += index
            average0 += flux[j,0]
            average1 += flux[j,1]
            
        hist[i,0] = average0 / bins
        hist[i,1] = average1 / bins

        index = j+1

    histo = hist.copy()
    avg = np.zeros([16,2])
    avg_new = np.zeros([17,2])
    for i in range(16):
        j = i*3
        avg[i,:] += (histo[j,:] + histo[j+1,:] + histo[j+2,:]) / 3

    avg_new[:,0] = np.insert(avg[:,0], 0, avg[0,0])
    avg_new[:,1] = np.insert(avg[:,1], 0, avg[0,1])
    return avg_new

In [186]:
def calculate_fission_source():
    # get values
    flux = tally1.cell_average_flux
    source = np.zeros(mesh1.Nx-1)
    dx = mesh1.dx
    index = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        nuSigmaF = material.nuSigmaF
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            source[j] = nuSigmaF[0]*flux[j,0] + nuSigmaF[1]*flux[j,1]
            
        index = j+1
        
    return source

In [187]:
def calculate_k(fission_new, fission_old):
    dx = mesh1.dx
    k_new = tally1.k * np.sum(dx*fission_new)/np.sum(dx*fission_old)
    return k_new

In [188]:
def calculate_source_S(g):
    # only has fission source in fast group
    if g == 0: 
        Sg = 1/tally1.k * source1.fission
    else:
        Sg = np.zeros_like(source1.fission)

    # get other values
    if g == 0:
        g_prime = 1
    else:
        g_prime = 0
    cell_average_flux = tally1.cell_average_flux[:,g_prime]
    dx = mesh1.dx
    index = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Sg[j] += SigmaS[g_prime, g] * cell_average_flux[j]
        index = j+1

    return Sg

In [189]:
def calculate_source_Q(Sg, g):
    # get values
    Qg = Sg.copy()
    cell_average_flux = tally1.cell_average_flux[:,g]
    dx = mesh1.dx
    index = 0

    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Qg[j] += SigmaS[g, g] * cell_average_flux[j]
        index = j+1
        

    return Qg/2

In [190]:
def outgoing_flux(Q, SigmaT, dx, u, angular_flux_inc):
    tau = SigmaT * dx / abs(u)
    edge_flux = angular_flux_inc * np.exp(-tau) + Q/SigmaT * (1 - np.exp(-tau))
    return edge_flux

In [191]:
def cell_average_flux(Q, SigmaT, dx, u, angular_flux_inc, angular_flux_exit):
    cell_flux = Q/SigmaT - (u * (angular_flux_exit - angular_flux_inc) / (dx * SigmaT))
    return cell_flux

In [192]:
def sweep(bc_incoming, Q, n, g):
    u = mesh1.U[n]
    w = mesh1.W[n]
    angular_flux_inc = bc_incoming

    # sweep right
    if u > 0:
        index = 0
        tally1.current[index, g] += w * u * angular_flux_inc
        tally1.cell_edge_flux[index, g] += w * angular_flux_inc
        # loop through all cells
        for i in range(len(cells)):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)
            

            # calculate source for current cell
            for j in range(bins):
                j += index
                
                # compute cell edge and cell average fluxes, save outgoing flux for next cell
                angular_flux_exit = outgoing_flux(Q[j], SigmaT, dx, u, angular_flux_inc)
                cell_average = cell_average_flux(Q[j], SigmaT, dx, u, angular_flux_inc, angular_flux_exit)
                angular_flux_inc = angular_flux_exit

                # tally flux and current
                tally1.cell_average_flux[j, g] += w * cell_average
                tally1.cell_edge_flux[j + 1, g] += w * angular_flux_exit
                tally1.current[j + 1, g] += w * u * angular_flux_exit
                

            index = j + 1
    
    # sweep left
    if u < 0:
        index = mesh1.Nx - 2
        tally1.current[index + 1, g] += w * u * angular_flux_inc
        tally1.cell_edge_flux[index + 1, g] += w * angular_flux_inc
        # loop through all cells
        for i in range(len(cells)-1,-1,-1):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)

            # calculate source for current cell
            for j in range(bins):
                j = index - j

                # compute cell edge and cell average fluxes, save outgoing flux for next cell
                angular_flux_exit = outgoing_flux(Q[j], SigmaT, dx, u, angular_flux_inc)
                cell_average = cell_average_flux(Q[j], SigmaT, dx, u, angular_flux_exit, angular_flux_inc)
                angular_flux_inc = angular_flux_exit 

                # tally flux and current
                tally1.cell_average_flux[j, g] += w * cell_average
                tally1.cell_edge_flux[j, g] += w * angular_flux_exit
                tally1.current[j, g] += w * u * angular_flux_exit

            index = j - 1

    return angular_flux_exit

Main loops

In [193]:
def source_iteration():
    iter = 100
    cell_average_flux_old = np.zeros_like(mesh1.mesh_flux)
    G = np.size(tally1.cell_average_flux, axis=1)

    for g in range(G):
        Sg = calculate_source_S(g)
        #print("SG: ", Sg)

        for i in range(iter):
            Qg = calculate_source_Q(Sg, g)
            #print("Q: ", Qg)
            tally1.zero_g(g)
            
            # sweep all angles
            for n in range(len(mesh1.U)):
                bc_in = np.flip(range(len(mesh1.U)))[n]
                bc1.bc[g, n] = sweep(bc1.bc[g, bc_in], Qg, n, g)
            
            # check convergence
            if si_convergence(tally1.cell_average_flux[:,g], cell_average_flux_old):
                break

            cell_average_flux_old = tally1.cell_average_flux[:,g].copy()
        print("Energy Group: ", g, "Source iterations: ", i)

In [194]:
def power_iteration():
    iter = 100
    k_old = 1
    f_old = np.ones_like(source1.fission)
    for i in range(iter):
        source_iteration()

        f_new = calculate_fission_source()
        source1.fission = f_new

        k_new = calculate_k(source1.fission, f_old)
        tally1.k = k_new

        if f_convergence(f_new, f_old) and k_convergence(k_new, k_old):
            break
        
        f_old = f_new.copy()
        k_old = k_new.copy()
        
        print("Power iteration: ", i)


In [195]:
def main_loop():
    power_iteration()
    print("k:", tally1.k)

Homogenized data

In [196]:
# materials
m1 = material(np.array([0.0,0.9]), 
              np.array([[0.2,0],[0.0,0.0]]), 
              np.array([0.2,0.6]))

water = material(np.array([0.0,0.0]), 
              np.array([[0.17,0.03],[0.0,1.1]]), 
              np.array([0.2,1.1]))

# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, m1), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(64 + 1, 10, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
                np.zeros((mesh1.Nx, 2)),
                np.zeros((mesh1.Nx, 2)),
                1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [197]:
main_loop()

Energy Group:  0 Source iterations:  99
Energy Group:  1 Source iterations:  42
Power iteration:  0
Energy Group:  0 Source iterations:  75
Energy Group:  1 Source iterations:  20
Power iteration:  1
Energy Group:  0 Source iterations:  4
Energy Group:  1 Source iterations:  3
Power iteration:  2
Energy Group:  0 Source iterations:  1
Energy Group:  1 Source iterations:  1
k: 1.499985585490304


In [198]:
fast_fr =  tally1.cell_edge_flux[64, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
fast_fl = tally1.cell_edge_flux[0, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
thermal_fr =  tally1.cell_edge_flux[64, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)
thermal_fl = tally1.cell_edge_flux[0, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)

print("Fast fR: ", fast_fr)
print("Fast fL: ", fast_fl)
print("Thermal fR: ", thermal_fr)
print("Thermal fL: ", thermal_fl)

Fast fR:  0.9956658312609054
Fast fL:  0.995667080134069
Thermal fR:  1.2222399436006912
Thermal fL:  1.2222430662266526


In [273]:
def homogenize_cs():
    # get values
    for g in range(2):
        print("group:", g)
        cell_average_flux = tally1.cell_average_flux[:,g]
        cs_sigmat = np.zeros_like(cell_average_flux)
        cs_sigmar = np.zeros_like(cell_average_flux)
        cs_nusigmaf = np.zeros_like(cell_average_flux)
        dx = mesh1.dx
        index = 0

        # loop through all cells
        for i in range(len(cells)):
            # get cell and material data
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            nuSigmaF = material.nuSigmaF[g]
            if g == 0:
                SigmaR = material.SigmaS[0,1]
            if g == 1:
                SigmaR = SigmaT - material.SigmaS[1,1]

            bins = int((right - left) / dx)
            
            # calculate source for current cell
            for j in range(bins):
                j += index
                #cs_sigmas[j] = SigmaS * cell_average_flux[j]
                cs_sigmat[j] = SigmaT * cell_average_flux[j]
                cs_sigmar[j] = SigmaR * cell_average_flux[j]
                cs_nusigmaf[j] = nuSigmaF * cell_average_flux[j]
            index = j+1
        
        cs_sigmat *= 1/(np.sum(cell_average_flux)/64)
        sigt = np.sum(cs_sigmat)/64
        D = 1/(3*sigt)
        print("D:", D)

        cs_nusigmaf *= 1/(np.sum(cell_average_flux)/64)
        nusigmaf = np.sum(cs_nusigmaf)/64
        print("nusigmaf:", nusigmaf)

        cs_sigmar *= 1/(np.sum(cell_average_flux)/64)
        sigr = np.sum(cs_sigmar)/64
        print("SigR:", sigr)
        
        if g == 0:
            sigsl = sigr.copy()
        if g == 1:
            print("SigSL: ", sigsl)
        print("")
    return 

In [274]:
homogenize_cs()

group: 0
D: 1.6666666666666665
nusigmaf: 0.0
SigR: 0.014956711204281319

group: 1
D: 0.37645110385642694
nusigmaf: 0.3861675105916773
SigR: 0.2574450070611182
SigSL:  0.014956711204281319



In [152]:
# materials
m2 = material(np.array([0.0,0.45]), 
              np.array([[0.185,0.015],[0.0,0.9]]), 
              np.array([0.2,1.2]))

water = material(np.array([0.0,0.0]), 
              np.array([[0.17,0.03],[0.0,1.1]]), 
              np.array([0.2,1.1]))

# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, m2), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(64 + 1, 10, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
                np.zeros((mesh1.Nx, 2)),
                np.zeros((mesh1.Nx, 2)),
                1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [153]:
main_loop()

Energy Group:  0 Source iterations:  99
Energy Group:  1 Source iterations:  95
Power iteration:  0
Energy Group:  0 Source iterations:  41
Energy Group:  1 Source iterations:  21
Power iteration:  1
Energy Group:  0 Source iterations:  1
Energy Group:  1 Source iterations:  1
k: 1.499994483156941


In [154]:
fast_fr =  tally1.cell_edge_flux[64, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
fast_fl = tally1.cell_edge_flux[0, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
thermal_fr =  tally1.cell_edge_flux[64, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)
thermal_fl = tally1.cell_edge_flux[0, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)

print("Fast fR: ", fast_fr)
print("Fast fL: ", fast_fl)
print("Thermal fR: ", thermal_fr)
print("Thermal fL: ", thermal_fl)

Fast fR:  0.9956597870125586
Fast fL:  0.9956607256389693
Thermal fR:  1.0827241883120133
Thermal fL:  1.0827235443898584


In [155]:
# materials
m2 = material(np.array([0.0,0.45]), 
              np.array([[0.185,0.015],[0.0,0.9]]), 
              np.array([0.2,1.2]))

m3 = material(np.array([0.0,0.345]), 
              np.array([[0.185,0.015],[0.0,0.9]]), 
              np.array([0.2,1.13]))

m4 = material(np.array([0.0,0.255]), 
              np.array([[0.185,0.015],[0.0,0.9]]), 
              np.array([0.2,1.07]))

water = material(np.array([0.0,0.0]), 
              np.array([[0.17,0.03],[0.0,1.1]]), 
              np.array([0.2,1.1]))

# cells
cells = []

for i in range(6):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, m2), cell(i+.9375, i+1.25, water)])

for i in range(6, 7):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, m3), cell(i+.9375, i+1.25, water)])

for i in range(7, 8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, m4), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(64 + 1, 10, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
                np.zeros((mesh1.Nx, 2)),
                np.zeros((mesh1.Nx, 2)),
                1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [156]:
main_loop()

Energy Group:  0 Source iterations:  99
Energy Group:  1 Source iterations:  99
Power iteration:  0
Energy Group:  0 Source iterations:  41
Energy Group:  1 Source iterations:  63
Power iteration:  1
Energy Group:  0 Source iterations:  24
Energy Group:  1 Source iterations:  41
Power iteration:  2
Energy Group:  0 Source iterations:  11
Energy Group:  1 Source iterations:  22
Power iteration:  3
Energy Group:  0 Source iterations:  5
Energy Group:  1 Source iterations:  8
Power iteration:  4
Energy Group:  0 Source iterations:  1
Energy Group:  1 Source iterations:  1
k: 1.500007802167878


In [157]:
fast_fr =  tally1.cell_edge_flux[64, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
fast_fl = tally1.cell_edge_flux[0, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
thermal_fr =  tally1.cell_edge_flux[64, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)
thermal_fl = tally1.cell_edge_flux[0, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)

print("Fast fR: ", fast_fr)
print("Fast fL: ", fast_fl)
print("Thermal fR: ", thermal_fr)
print("Thermal fL: ", thermal_fl)

Fast fR:  0.9808866107284445
Fast fL:  0.9989963523104707
Thermal fR:  1.3330566397574428
Thermal fL:  0.9830124900362326


In [164]:
# materials
u1 = material(np.array([0.0,0.3]), 
              np.array([[0.2,0],[0,0]]), 
              np.array([0.2,0.2]))

water = material(np.array([0.0,0.0]), 
              np.array([[0.17,0.03],[0.0,1.1]]), 
              np.array([0.2,1.1]))

# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, u1), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(64 + 1, 10, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
                np.zeros((mesh1.Nx, 2)),
                np.zeros((mesh1.Nx, 2)),
                1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [165]:
main_loop()

Energy Group:  0 Source iterations:  99
Energy Group:  1 Source iterations:  84
Power iteration:  0
Energy Group:  0 Source iterations:  75
Energy Group:  1 Source iterations:  36
Power iteration:  1
Energy Group:  0 Source iterations:  3
Energy Group:  1 Source iterations:  3
Power iteration:  2
Energy Group:  0 Source iterations:  1
Energy Group:  1 Source iterations:  1
k: 1.4999803656043504


In [166]:
fast_fr =  tally1.cell_edge_flux[64, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
fast_fl = tally1.cell_edge_flux[0, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
thermal_fr =  tally1.cell_edge_flux[64, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)
thermal_fl = tally1.cell_edge_flux[0, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)

print("Fast fR: ", fast_fr)
print("Fast fL: ", fast_fl)
print("Thermal fR: ", thermal_fr)
print("Thermal fL: ", thermal_fl)

Fast fR:  0.9956458169396128
Fast fL:  0.9956470906674258
Thermal fR:  1.0827951081464646
Thermal fL:  1.0827982606952136


In [170]:
.3/2*(np.sum(tally1.cell_average_flux[:,1])) /np.sum(tally1.cell_average_flux[:,1]) 

0.15

In [161]:
# materials
u2 = material(np.array([0.0,0.15]), 
              np.array([[0.185,0.015],[0.0,0.9]]), 
              np.array([0.2,1.0]))

water = material(np.array([0.0,0.0]), 
              np.array([[0.17,0.03],[0.0,1.1]]), 
              np.array([0.2,1.1]))

# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, u2), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(64 + 1, 10, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
                np.zeros((mesh1.Nx, 2)),
                np.zeros((mesh1.Nx, 2)),
                1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [162]:
main_loop()

Energy Group:  0 Source iterations:  99
Energy Group:  1 Source iterations:  99
Power iteration:  0
Energy Group:  0 Source iterations:  41
Energy Group:  1 Source iterations:  99
Power iteration:  1
Energy Group:  0 Source iterations:  11
Energy Group:  1 Source iterations:  41
Power iteration:  2
Energy Group:  0 Source iterations:  5
Energy Group:  1 Source iterations:  8
Power iteration:  3
Energy Group:  0 Source iterations:  1
Energy Group:  1 Source iterations:  1
k: 1.4999801469021337


In [163]:
fast_fr =  tally1.cell_edge_flux[64, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
fast_fl = tally1.cell_edge_flux[0, 0] / (np.sum(tally1.cell_average_flux[:,0]) / 64)
thermal_fr =  tally1.cell_edge_flux[64, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)
thermal_fl = tally1.cell_edge_flux[0, 1] / (np.sum(tally1.cell_average_flux[:,1]) / 64)

print("Fast fR: ", fast_fr)
print("Fast fL: ", fast_fl)
print("Thermal fR: ", thermal_fr)
print("Thermal fL: ", thermal_fl)

Fast fR:  0.9956465679245036
Fast fL:  0.9956505652295919
Thermal fR:  1.0284900592358555
Thermal fL:  1.0285049626366953
